In [ ]:
#下载微信文章到数据库，批量版
#v2.0.1 2018.12.3-16.01 添加新文章

In [116]:
# 函数库
import re, requests, sys, pymysql


def getInfoByURL(url):
    #####################################
    #1.爬虫获取信息
    #####################################
    # 发出请求
    respose=requests.get(msg_url)
    assert respose.status_code==200,'Error: requests.get'
    #print(respose.text)

    #1.msg_title
    title=re.findall(r'msg_title = "(.*?)"',respose.text,re.S)[0]
    #print('title=',title)

    #2.msg_desc
    msg_desc=re.findall(r'msg_desc = "(.*?)"',respose.text,re.S)[0]
    #print('msg_desc=',msg_desc)

    #3.js_content
    rs=re.findall(r'id="js_content"\>(.*?)\<\/div\>',respose.text,re.S)[0]
    #print('rs=', rs)


    #4.获取日期
    #aid1 由第一行获取数组：日期字符串，单词数
    def getDateStrSize(strs):
        d1=strs   #rs[1:300];d1
        d2=re.split("words",d1)[0]
        d3=re.split(">",d2)[-1]
        d4=re.split("\|", d3)

        return [d4[0].strip(), int(d4[1].strip())] ##'Aug 30, 2018'日期; 859 单词数
    d_n=getDateStrSize(rs[1:300]);
    #print(d_n)  ##['Aug 30, 2018', 859]

    #aid2 由日期字符串'Aug 30, 2018'获得日期数字字符串 '20180830'
    def getDateFromStr(s):
        a = s; #d4[0].strip()
        month=a[0:3]; month
        year=a[-4:];year
        day=re.split("[\s,]",a)[1];day
        #
        m_dict={"Jan":"01", "Feb":"02", "Mar":"03", "Apr":"04", "May":"05", "Jun":"06", 
                "Jul":"07", "Aug":"08", "Sep":"09", "Oct":"10", "Nov":"11", "Dec":"12" }
        #
        def to2(day):
            if len(day)==1:
                day="0"+day
            return day;

        return year+m_dict[month]+to2(day)

    dateN=getDateFromStr(d_n[0]) #'20180830'

    #5.获取字数
    size=d_n[1]
    #print(dateN, size)
    
    
    #写入文件
    with open('/home/wangjl/web/en'+dateN+'.html','w') as f:
        f.write('<meta http-equiv="Content-Type" content="text/html; charset=utf-8">')
        f.write(title+"<br>")
        f.write(msg_desc)
        f.write(rs)
    print("Title: ", title)
    print("write in to file: http://y.biomooc.com/wangjl/en"+dateN+".html")
    
    return [url,dateN, size,title,msg_desc,rs]


    
#####################################
#2.写入数据库
#####################################

#db:wang
#CREATE TABLE `msg_English` (
#	`msg_id` int(4) not null primary key auto_increment,
#  `msg_url` varchar(300) , #url
#	
#  `msg_origin` char(100) NOT NULL DEFAULT '', #来源
#  `msg_date` char(50) NOT NULL DEFAULT '', #日期
#  `msg_size` int(10) NOT NULL DEFAULT 0, #字数
#
#  `msg_title` char(100) NOT NULL DEFAULT '', 
#  `msg_desc` char(150) NOT NULL DEFAULT '',
#  `msg_content` text #PRIMARY KEY (`msg_origin`,`msg_date`)
#) ENGINE=MyISAM DEFAULT CHARSET=utf8 COLLATE=utf8_bin COMMENT='微信英文文章表';

def insertInfo2DB(info,source="英语读报"):
    # 打开数据库连接
    db = pymysql.connect(host='y.biomooc.com',port=7070,user='root', password='123456', database='wang')

    # 使用cursor()方法获取操作游标 
    cursor = db.cursor()




    #如果没有，则保存到数据库中
    def DoInsertDB(info,source):
        # SQL 插入语句
        sql = "INSERT INTO msg_English(msg_url,msg_origin,msg_date, msg_size, msg_title, msg_desc,msg_content) \
               VALUES ('%s', '%s', '%s', '%d', '%s', '%s','%s' )" % \
              (info[0],source, info[1],info[2],info[3],info[4],pymysql.escape_string(info[5]))
                #[url,dateN, size,title,msg_desc,rs]
               #(msg_url, '英语读报', dateN, size, title, msg_desc, pymysql.escape_string(rs))
        try:
            cursor.execute(sql) # 执行sql语句
            db.commit() # 提交到数据库执行
            print("commit successfully")
        except:
            rsponse=db.rollback() # 发生错误时回滚
            print("rollback")
            print( sys.exc_info() )

    #如果日期已经存在，则不进行处理


    # SQL 查询语句
    dateN2=info[1]
    #dateN2="20170102"
    sql = 'select msg_date from msg_English where msg_date like "'+dateN2+'";'

    # 执行sql语句
    cursor.execute(sql)
    values = cursor.fetchall()
    if len(values)==0:
        print("Not exist the date")
        DoInsertDB(info,source)
    else:
        print("The date "+dateN2+" has already existed.")

    # 关闭数据库连接
    db.close()

print("=====1.end====")

=====1.end====


In [117]:
#预备 测试url 获取

#如何批量获取微信文章的url?
import requests
from lxml import html

#1.请求数据

###############
#1.手工访问该页面
# https://weixin.sogou.com/weixin?type=2&s_from=input&query=英语读报&ie=utf8&_sug_=n&_sug_type_= 
# 右侧点击公众号
#2. 然后F12，把html全部从控制台复制出来。
#3. cat命令保存到/home/wangjl/web/urls.html文件中。 
###############

urls="http://y.biomooc.com/wangjl/urls.html"
page=requests.Session().get(urls)
#print("==ok  http://y.biomooc.com/wangjl/urls.html")


#2.解析数据：获取批量URL
tree=html.fromstring(page.text)

result=tree.xpath('//@hrefs')#重复出现2次，所以只取奇数即可。
arrURL=[]
for i in range( len(result) ):
    if i%2==0:
        arrURL.append("https://mp.weixin.qq.com"+result[i])
print(len(arrURL))
print(arrURL[0])

print("="*20, "预期会有10个url，请手工测试一个url能否打开。")

10
https://mp.weixin.qq.com/s?timestamp=1543823695&src=3&ver=1&signature=pGFsXhjV7IozHY5PyM1RMbbtB14HwUxvHw3drWTnrG7TZFZle6E1jLkJxWsaGOJ4rmIWW1iQRGQz6eoUJjckpy145UgxG0H9aJU3fbqlPIMFT7RXGlOYIJs4HTTY0vH9PsJIrZK1UflXmSM96l9NKsCXOtzUtxDC4UUGzcoJLZk=
==================== 预期会有10个url，请手工测试一个url能否打开。


In [118]:
#批量实施：下载并保存到文件和数据库
import requests
from lxml import html

#1.请求数据

#
#
###############
#1.手工访问该页面
# https://weixin.sogou.com/weixin?type=2&s_from=input&query=英语读报&ie=utf8&_sug_=n&_sug_type_= 右侧点击公众号
#2. 然后F12，把html全部从控制台复制出来。
#3. cat命令保存到/home/wangjl/web/urls.html文件中。 
###############
urls="http://y.biomooc.com/wangjl/urls.html"
page=requests.Session().get(urls)
#print("==ok  http://y.biomooc.com/wangjl/urls.html")


#2.解析数据：获取批量URL
tree=html.fromstring(page.text)
#result=tree.xpath('//ul/li[contains(@class,"ui-slide-item")]/attribute::data-title')#hit 32 items 

result=tree.xpath('//@hrefs')#重复出现2次，所以只取奇数即可。
arrURL=[]
for i in range( len(result) ):
    if i%2==0:
        arrURL.append("https://mp.weixin.qq.com"+result[i])
#print(len(arrURL))
#arrURL[0]



#3. 批量URL获取文章，并插入数据库
import time,random
for i in range(len(arrURL)): #随机时间段后[2s,12s]执行
#for i in range(3):    
    msg_url=arrURL[i]
    print(msg_url)
    #2.get Info  return [url,dateN, size,title,msg_desc,rs]
    info= getInfoByURL(msg_url)

    #3.插入数据库
    insertInfo2DB(info)
    
    #等待随机时间
    pause=2+10*random.random()
    print(str(i)+' '+str(pause))
    time.sleep(pause)

print("\n=======================2.end")

https://mp.weixin.qq.com/s?timestamp=1543823695&src=3&ver=1&signature=pGFsXhjV7IozHY5PyM1RMbbtB14HwUxvHw3drWTnrG7TZFZle6E1jLkJxWsaGOJ4rmIWW1iQRGQz6eoUJjckpy145UgxG0H9aJU3fbqlPIMFT7RXGlOYIJs4HTTY0vH9PsJIrZK1UflXmSM96l9NKsCXOtzUtxDC4UUGzcoJLZk=
Title:  economist|Chip wars: China, America and silicon supremacy
write in to file: http://y.biomooc.com/wangjl/en20181203.html
Not exist the date
commit successfully
0 6.819681301416124
https://mp.weixin.qq.com/s?timestamp=1543823695&src=3&ver=1&signature=pGFsXhjV7IozHY5PyM1RMbbtB14HwUxvHw3drWTnrG7TZFZle6E1jLkJxWsaGOJ4rmIWW1iQRGQz6eoUJjckp1zsDlNmIuiEeBpapVZIVPhM9yMEfV9QiRRe9mAtSmrEiU8bUPXV-XdEzQPa3IgHUWLBAkG0TOtofUgOuX7rlSY=
Title:  Time|The True Story Behind the Vincent van Gogh Biopic At ...
write in to file: http://y.biomooc.com/wangjl/en20181202.html
Not exist the date
commit successfully
1 2.711133284607418
https://mp.weixin.qq.com/s?timestamp=1543823695&src=3&ver=1&signature=pGFsXhjV7IozHY5PyM1RMbbtB14HwUxvHw3drWTnrG7TZFZle6E1jLkJxWsaGOJ4rm

In [87]:
# 单个URL的下载和插入数据库
#实施步骤 

#1.set URL
#msg_url='https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485084&idx=1&sn=70239065aaa1f59610dab9894df7f745'
msg_url=arrURL[9]
print(msg_url)

#2.get Info  return [url,dateN, size,title,msg_desc,rs]
info= getInfoByURL(msg_url)

#3.插入数据库
insertInfo2DB(info)

https://mp.weixin.qq.com/s?timestamp=1542804854&src=3&ver=1&signature=9pNdKe*6eoeW2LOZb*bLOQhkhDUBekOk6xuXdQ0221KhpCOKq6KfLSLsqmIs9mxpGD2LptVSykNib2pDxSyZCRAawV0nm6atfH6NvWa8NKyOsAYp1YmA2wrC2o9qArBi8hsQtoEzJMH9korF7tD9pVNAXjK*IlRKV9rvGFpe7IQ=
write in to file: http://y.biomooc.com/wangjl/en20181112.html
Not exist the date
commit successfully


In [119]:
# 获取mysql已有文章的目录

#!/usr/bin/python
# -*- coding: UTF-8 -*-
import pymysql

#mysql 获取数据

# 打开数据库连接
db = pymysql.connect(host='y.biomooc.com',port=7070,user='root', password='123456', database='wang')

# 使用cursor()方法获取操作游标 
cursor = db.cursor()

# SQL 插入语句
#sql = "select msg_date,msg_title,msg_size from msg_English order by msg_size DESC" #按照字数倒序
sql = "select msg_origin,msg_date,msg_title,msg_size from msg_English order by msg_date"

# 执行sql语句
cursor.execute(sql)
values = cursor.fetchall()

for i in range(len(values)):
    print( list(values[i]) )

# 关闭数据库连接
db.close()

['英语读报', '20180830', 'Time|A New Study Says Any Amount of Drinking Is Bad for You', 859]
['英语读报', '20181112', 'cnbc | Mark says it’s time to buy stocks in emerging markets', 570]
['英语读报', '20181113', 'HWR|Stan Lee, Marvel Comics&#39; Real-Life Superhero, Dies at 95', 1501]
['英语读报', '20181114', 'nature|China’s crackdown on genetics breaches could deter data..', 945]
['英语读报', '20181115', 'newRepublic | Stan Lee: the Midwife of the Marvel Universe', 1065]
['英语读报', '20181116', 'economist | Automation in Japan: Human endurance', 527]
['英语读报', '20181117', 'cnn | Which country has the best food?', 451]
['英语读报', '20181118', 'Fox | Google replaces CEO of cloud unit with former Oracle exec', 231]
['英语读报', '20181119', 'economist | China’s internet titan has had a bruising 2018', 1076]
['英语读报', '20181120', 'science|Large, strangely dim galaxy found lurking on far side...', 583]
['英语读报', '20181121', 'economist | Universities’ finances: After the boom, the bust', 803]
['英语读报', '20181122', 'guardian|

In [124]:
# 根据关键词，搜索mysql中匹配的例句

#!/usr/bin/python
# -*- coding: UTF-8 -*-
import pymysql,re

# keyword
#keyword="caveat"
#keyword="moderate"
keyword="overall"

#mysql 获取数据

# 打开数据库连接
db = pymysql.connect(host='y.biomooc.com',port=7070,user='root', password='123456', database='wang')

# 使用cursor()方法获取操作游标 
cursor = db.cursor()

# SQL 插入语句
#sql = "select msg_date,msg_title,msg_size from msg_English order by msg_size DESC" #按照字数倒序
sql = "select msg_origin,msg_date,msg_title,msg_size,msg_content from msg_English where msg_content like '%" +keyword+ "%' order by msg_date"

# 执行sql语句
cursor.execute(sql)
values = cursor.fetchall()

print('hit', len(values), 'item(s): ', keyword )

counter=0
for i in range(len(values)):
    date=values[i][1]
    title=values[i][2]
    size=values[i][3]
    text=re.sub(r'<[^>]+>', "", values[i][4])
    
    print('='*40,'\n',date, title,"<",size,'> words')
    
    arr=re.split("[.?\n]",text) #todo 怎么保留句号呢？
    for j in range( len(arr) ):
        if len( re.findall(keyword,arr[j]) )>0:
            counter+=1
            print(counter,"| ",arr[j],"\n",sep="")


# 关闭数据库连接
db.close()

print("==end==")

hit 2 item(s):  overall
 20180830 Time|A New Study Says Any Amount of Drinking Is Bad for You < 859 > words
1| A new study concludes there’s no amount of alcohol consumption that’s safe for overall health — a finding that’s likely to surprise moderate drinkers, and that has left some experts unconvinced

2| “We don’t have the information for specific individuals…we’re making overall recommendations at the population level,” she says

3|  “If you’re running a health system in a country, it’s better overall for the population of your country to not drink at all than to drink a little bit

4| “I think this is actually consistent with every organization’s recommendation that, overall, no one should start drinking to prevent heart disease or diabetes,” Mozaffarian says

 20181114 nature|China’s crackdown on genetics breaches could deter data.. < 945 > words
5|  Although the penalties set back certain research projects, the companies say that overall they have not had a large impact on their